In [1]:
import pandas as pd
import sys
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import statistics
import math 
from itertools import repeat
from collections import Counter

In [2]:
def genNewFeatures(data):

  temp = data

  temp = getSpread(temp)
  temp = getAWS(temp)
  temp = anomalyDetect(temp)
  temp = spreadChange(temp)
  temp = imbalance(temp)
  temp = VPIN(temp).drop(columns = 'Bucket #')

  return temp

In [3]:
def getSpread(data):

  data['Spread'] = data['Best_Offer_Price'] - data['Best_Bid_Price']
  return data

In [4]:

def getAWS(data,time = "minutes"):

    if time == "minutes":
      n = 5
    else:
      n = 300 # seconds

    # Input quote data and n, the number of look back minutes

    TBCS = []
    # Get consolidated bid size for past 'n' minutes
    for i in range(0,len(data)):
      if i < n:
        sum = int(data.loc[0:i,['FB0']].sum())
      else:
        sum = int(data.loc[i-n:i,['FB0']].sum())
      TBCS.append(sum)
    
    TACS = []
    # Get consolidated ask size for past 'n' minutes
    for i in range(0,len(data)):
      if i < n:
        sum = int(data.loc[0:i,['FA0']].sum())
      else:
        sum = int(data.loc[i-n:i,['FA0']].sum())
      TACS.append(sum)

    WBP = []
    WAP = []
    # Get weighted bid price and weighted ask price 
    for i in range(0,len(TBCS)):
      if i < n:
        CBS = data.loc[0:i,['FB0']] # vol
        BP = data.loc[0:i,['Best_Bid_Price']] # price
        CBS = CBS.FB0.tolist()
        BP = BP.Best_Bid_Price.tolist()

        ABS = data.loc[0:i,['FA0']] # vol
        AP = data.loc[0:i,['Best_Offer_Price']] # price
        ABS = ABS.FA0.tolist()
        AP = AP.Best_Offer_Price.tolist()

      else: 

        CBS = data.loc[i-n:i,['FB0']] # vol
        BP = data.loc[i-n:i,['Best_Bid_Price']] # price
        CBS = CBS.FB0.tolist()
        BP = BP.Best_Bid_Price.tolist()

        ABS = data.loc[i-n:i,['FA0']] # vol
        AP = data.loc[i-n:i,['Best_Offer_Price']] # price
        ABS = ABS.FA0.tolist()
        AP = AP.Best_Offer_Price.tolist()

      wbp = np.dot(CBS,BP)/ TBCS[i]
      WBP.append(wbp)

      wap = np.dot(ABS,AP)/ TACS[i]
      WAP.append(wap)

    AWS = [WAP[i] - WBP[i] for i in range(0,len(WAP))]

    data['AWS'] = AWS
    return data


In [5]:
def anomalyDetect(data,time = "minutes"):

    if time == "minutes":
      n = 10
    else: 
      n = 600 # 600 seconds = 10 minutes

    anom = []

    for i in range(0,len(data)):

      if i < n: 
        nums = data.loc[0:i,['AWS']]
        nums = nums.AWS.tolist() 
        mean = np.mean(nums)
        std = np.std(nums)
        size = i+1

      else: 
        nums = data.loc[i-n:i,['AWS']]
        nums = nums.AWS.tolist() 
        mean = np.mean(nums)
        std = np.std(nums)
        size = n

      z = 1.96 # default 
      up = mean + (z*std/np.sqrt(size))
      low = mean - (z*std/np.sqrt(size))

      curr = float(data.loc[i,['AWS']])
      if curr > up or curr < low:
        anom.append(1) # 1 = yes
      else:
        anom.append(0) # 0 = no 
    
    data['Anomaly'] = anom
    return data


In [6]:
def spreadChange(data):

  nums = [0]

  for i in range(1,len(data)):
    if float(data.loc[i,['Spread']]) >= float(data.loc[i-1,['Spread']]): # spread increase/stayed same
      nums.append(1)
    else:
      nums.append(-1)
  data['Spread Change'] = nums
  return data

In [7]:
def imbalance(data):

  imb = np.subtract(data['FB0'].tolist(), data['FA0'].tolist()) / np.add(data['FB0'].tolist(), data['FA0'].tolist())
  data['Quote IMB'] = imb
  return data

In [8]:

def VPIN(data,time = "minutes"):

  V = math.floor(len(data) / 72) # volume per bucket, num is arbitrary, this is what paper used
  L = 5 # number of buckets to look back, arbitrary, this means 5 minutes, for second data would be 300

  if time == "minutes":
    L = 5
  else: 
    L = 300

  buy = [] # classify as buy or sell (here, bid or ask)
  sell = []

  for i in range(0,len(data)):

      bidVol = float(data.loc[i,['FB0']])
      askVol = float(data.loc[i,['FA0']])

      buy.append(list(np.repeat(1,bidVol)))
      buy.append(list(np.repeat(0,askVol)))

      sell.append(list(np.repeat(0,bidVol)))
      sell.append(list(np.repeat(1,askVol)))
  
  buy = [item for sublist in buy for item in sublist]
  sell = [item for sublist in sell for item in sublist] 

  buckets = [] # holds buckets
  i = 1 # counter 

  while True: 

    # Partition
    currBuys = buy[(i-1)*V:i*V] 
    currSells = sell[(i-1)*V:i*V]

    vB = Counter(currBuys)[1] # Counts number of bids in given bucket
    vS = Counter(currSells)[1] # Counts number of asks in given bucket

    buckets.append((vB,vS)) # Add this "bucket" to list of buckets
    if vB + vS < V: # This means we've reached the end
      break
    i += 1


  # Now, we look at the imbalance for the past L buckets, and take abs value. 

  # This stores the imbalance value of each bucket
  # For a given instance, we look back L buckets, and sum these values. 
  vals = [] 

  for i in range(0,len(buckets)):
    temp = buckets[i]
    diff = abs(temp[0] - temp[1])
    val = round(diff / (L*V), 5)
    vals.append(val)

  buck = 1 # Buckets from 1...n
  currBucket = []
  i = 0

  while True:
    sum = 0
    while sum < V and i < len(data):
      bidVol = float(data.loc[i,['FB0']])
      askVol = float(data.loc[i,['FA0']])
      sum += bidVol + askVol
      currBucket.append(buck)
      i+= 1
    buck += 1
    if i >= len(data):
      break

  data['Bucket #'] = currBucket # indicates which bucket a quote is in 

  VPIN = []
  for i in range(0,len(data)):
    currBucket = int(data.loc[i,['Bucket #']])

    if currBucket < L:
      currVPIN = math.fsum(vals[0:currBucket])
      VPIN.append(round(currVPIN,5))
    else:
      currVPIN = math.fsum(vals[currBucket - L - 1:currBucket-1])
      VPIN.append(round(currVPIN,5))

  data['VPIN'] = VPIN
  data.drop(columns = "Bucket #")

  return data


In [9]:
quotes=pd.read_csv("labeled_data.csv")

In [10]:
quotes.head()

,last_interval,Exchange,Symbol,Best_Bid_Price,FB0,Best_Offer_Price,FA0,FB2,FA2,p_time,Next_Best_Bid,Next_Best_Offer,outcome
0,2020-01-02 09:30:00,P,AAPL,296.24,2.0,296.29,1.0,-1.0,0.0,2020-01-02 09:30:00.134062,295.71,295.78,-1
1,2020-01-02 09:31:00,Q,AAPL,295.71,1.0,295.78,2.0,0.0,0.0,2020-01-02 09:31:00.000691,295.49,295.52,-1
2,2020-01-02 09:32:00,Z,AAPL,295.49,3.0,295.52,2.0,0.0,1.0,2020-01-02 09:32:00.019756,295.68,295.73,1
3,2020-01-02 09:33:00,Z,AAPL,295.68,1.0,295.73,2.0,-1.0,0.0,2020-01-02 09:33:00.000277,296.53,296.60,1
4,2020-01-02 09:34:00,J,AAPL,296.53,3.0,296.60,4.0,0.0,1.0,2020-01-02 09:34:00.114073,296.90,296.93,1


In [11]:
newData = genNewFeatures(quotes)

In [12]:
newData.head()

,last_interval,Exchange,Symbol,Best_Bid_Price,FB0,Best_Offer_Price,FA0,FB2,FA2,p_time,Next_Best_Bid,Next_Best_Offer,outcome,Spread,AWS,Anomaly,Spread Change,Quote IMB,VPIN
0,2020-01-02 09:30:00,P,AAPL,296.24,2.0,296.29,1.0,-1.0,0.0,2020-01-02 09:30:00.134062,295.71,295.78,-1,0.05,0.050000,0,0,0.333333,0.02727
1,2020-01-02 09:31:00,Q,AAPL,295.71,1.0,295.78,2.0,0.0,0.0,2020-01-02 09:31:00.000691,295.49,295.52,-1,0.07,-0.113333,0,1,-0.333333,0.02727
2,2020-01-02 09:32:00,Z,AAPL,295.49,3.0,295.52,2.0,0.0,1.0,2020-01-02 09:32:00.019756,295.68,295.73,1,0.03,0.001333,0,-1,0.200000,0.02727
3,2020-01-02 09:33:00,Z,AAPL,295.68,1.0,295.73,2.0,-1.0,0.0,2020-01-02 09:33:00.000277,296.53,296.60,1,0.05,0.001429,0,1,-0.333333,0.02727
4,2020-01-02 09:34:00,J,AAPL,296.53,3.0,296.60,4.0,0.0,1.0,2020-01-02 09:34:00.114073,296.90,296.93,1,0.07,0.075182,1,1,-0.142857,0.02727
